In [128]:
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
from torch import nn
import torch
import numpy as np
from torch.autograd import Variable 
from sys import *
from subprocess import *
import os


In [129]:
mobi_data = pd.read_csv("../data/Mobi_System_Data_2020.csv", compression='zip').dropna()
geo_data = pd.read_csv('../data/geocodings.csv',index_col=0)
data = mobi_data.merge(geo_data.rename({'lat':'Departure lat','long':'Departure long'},axis=1),left_on='Departure station',right_on='address').drop('address',axis=1)
data = data.merge(geo_data.rename({'lat':'Return lat','long':'Return long'},axis=1),left_on='Return station',right_on='address').drop('address',axis=1)
print(data.columns)
data["time"] = pd.to_datetime(data["Departure"], format="%Y-%m-%d %H:%M")
# data["hour"] = data["time"].dt.hour
# data["day"] = data["time"].dt.day
# data["dayofweek"] = data["time"].dt.dayofweek
# data["month"] = data["time"].dt.month
# data["year"] = data["time"].dt.year
stations = pd.read_csv("../data/stations.csv")
max_lat, min_lat = stations["lat"].max(), stations["lat"].min()
max_long, min_long = stations["long"].max(), stations["long"].min()

Index(['Unnamed: 0', 'Departure', 'Return', 'Bike', 'Departure station',
       'Return station', 'Membership type', 'Covered distance (m)',
       'Duration (sec.)', 'Departure battery voltage (mV)',
       'Return battery voltage (mV)', 'Departure temperature (C)',
       'Return temperature (C)', 'Stopover duration (sec.)',
       'Number of stopovers', 'postal_code_x', 'Departure lat',
       'Departure long', 'postal_code_y', 'Return lat', 'Return long'],
      dtype='object')


In [130]:
def generate_time_df(start_time, period):
    # predict_data = pd.DataFrame(data={
    #     "time": [],
    #     "lat": [],
    #     "long": [],
    #     "parks": [],
    #     "stops": [],
    #     "population": [],
    #     "station": []})
    # for row in stations.iterrows():
    #     time = pd.date_range(start_time, periods=peroid, freq='H')

    #     station_data = pd.DataFrame(data={"time": time})
    #     station_data["lat"] = (row[1]["lat"] - min_lat) / (max_lat - min_lat)
    #     station_data["long"] = (row[1]["long"] - min_long) / (max_long - min_long)
    #     station_data["station"] = row[1]["station"]
    #     station_data["parks"] = row[1]["parks"]
    #     station_data["stops"] = row[1]["stops"]
    #     station_data["population"] = row[1]["population"]
    #     station_data["bike_ways"] = row[1]["bike_ways"]
        
    #     predict_data = pd.concat([predict_data, station_data])
    time_series = pd.date_range(start_time, periods=period, freq='H')
    predict_data = pd.DataFrame(data={"time": time_series})
    predict_data = predict_data.merge(stations, how="cross")
    predict_data["lat"] = (predict_data["lat"] - min_lat) / (max_lat - min_lat)
    predict_data["long"] = (predict_data["long"] - min_long) / (max_long - min_long)
    
    return predict_data


In [131]:
features = ["month", "dayofweek", "hour", "lat", "long", "parks", "stops", "population", "bike_ways"]
counts_data_departure = data.groupby(["time", "Departure lat", "Departure long", "Departure station"]).size().reset_index(name='counts')
counts_data_departure = counts_data_departure.rename(columns={"Departure lat": "lat", "Departure long": "long", "Departure station": "station"})

departure_time_df = generate_time_df('2020-01-01', 8760)

train_data_departure = departure_time_df.merge(
    counts_data_departure[["time", "station", "counts"]], 
    how="left", 
    on=["time", "station"])
train_data_departure["counts"] = train_data_departure["counts"].fillna(0.00001)

# train_data_departure["month"] = train_data_departure["month"] / 12
# train_data_departure["day"] = train_data_departure["day"] / 7
# train_data_departure["hour"] = train_data_departure["hour"] / 23
    



In [132]:
counts_data_return = data.loc[:, ["time", "Return lat", "Return long", "Return station", "Duration (sec.)"]].rename(columns={"Duration (sec.)": "duration"})
counts_data_return["duration"] = (counts_data_return["duration"] / 3600).astype(int)
counts_data_return["time"] = counts_data_return["time"] + pd.to_timedelta(counts_data_return["duration"], unit="H")
# counts_data_return["Duration (sec.)"] = (counts_data_return["Duration (sec.)"] / 3600).astype(int)
# counts_data_return["hour"] = counts_data_return["hour"] + counts_data_return["Duration (sec.)"]
# counts_data_return["day"] = counts_data_return["day"] + (counts_data_return["hour"] / 24).astype(int)
# counts_data_return["dayofweek"] = counts_data_return["dayofweek"] + (counts_data_return["hour"] / 24).astype(int)
# counts_data_return["hour"] = counts_data_return["hour"].mod(24)
counts_data_return = counts_data_return.drop(columns=["duration"])
counts_data_return = counts_data_return.groupby(["time", "Return lat", "Return long", "Return station"]).size().reset_index(name='counts')
counts_data_return = counts_data_return.rename(columns={"Return lat": "lat", "Return long": "long", "Return station": "station"})
return_time_df = generate_time_df('2020-01-01', 8760)
train_data_return = return_time_df.merge(
    counts_data_return[["time", "station", "counts"]], 
    how="left", 
    on=["time", "station"])
train_data_return["counts"] = train_data_return["counts"].fillna(0.00001)

# train_data_return["month"] = train_data_return["month"] / 12
# train_data_return["day"] = train_data_return["day"] / 7
# train_data_return["hour"] = train_data_return["hour"] / 23

In [133]:
print(train_data_return["counts"].sum())
print(train_data_departure["counts"].sum())

549209.2231999997
549864.2163500001


In [134]:
train_data = train_data_departure.loc[:, ["time", "station", "counts"]].rename(columns={"counts": "counts_departure"})
train_data["counts_return"] = train_data_return["counts"]
# train_data["ratio"] = train_data_departure["counts"] / train_data_return["counts"]
train_data["hour"] = train_data["time"].dt.hour

conditions = [
    (train_data["hour"] <= 6) & (train_data["hour"] >= 22),
    (train_data["hour"] >= 7) & (train_data["hour"] <= 11),
    (train_data["hour"] >= 12) & (train_data["hour"] <= 21)
]
choices = [0, 0.5, 1]
train_data["hour"] = np.select(conditions, choices, default=0)
train_data["month"] = train_data["time"].dt.month
train_data["day"] = train_data["time"].dt.day
train_data["dayofweek"] = train_data["time"].dt.dayofweek
train_data = train_data.groupby(["month", "day", "dayofweek", "hour", "station"]).sum().reset_index()
train_data = train_data.merge(stations, on="station", how="left")
train_data["lat"] = (train_data["lat"] - min_lat) / (max_lat - min_lat)
train_data["long"] = (train_data["long"] - min_long) / (max_long - min_long)
train_data["ratio"] = train_data["counts_departure"] / train_data["counts_return"]
# print(train_data)
# train_data = train_data.groupby(["month", "dayofweek", "day", "hour", "lat", "long", "parks", "stops", "population", "bike_ways"]).sum()
# print(train_data["hour"])

conditions = [
    train_data["ratio"] <= 0.5,
    (train_data["ratio"] > 0.5) & (train_data["ratio"] < 2.0),
    train_data["ratio"] >= 2.0
]
choices = [0, 1, 2]
train_data["class"] = np.select(conditions, choices, default=1)
train_data["month"] = train_data["month"] / 12
train_data["dayofweek"] = train_data["dayofweek"] / 6
# train_data["hour"] = train_data["hour"] / 23

num_1 = train_data.loc[train_data["class"] == 0].shape[0]
num_2 = train_data.loc[train_data["class"] == 1].shape[0]
num_3 = train_data.loc[train_data["class"] == 2].shape[0]
x = train_data[features]
y = train_data["class"]
print(num_1, num_2, num_3)

34949 157146 37855


In [135]:
from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss")
model = model.fit(x, y)

In [150]:
time_series = pd.date_range('2022-05-01', periods=168, freq='H')
prediction = pd.DataFrame(data={"time": time_series})

prediction["hour"] = prediction["time"].dt.hour

conditions = [
    (prediction["hour"] <= 6) & (prediction["hour"] >= 22),
    (prediction["hour"] >= 7) & (prediction["hour"] <= 11),
    (prediction["hour"] >= 12) & (prediction["hour"] <= 21)
]
choices = [0, 0.5, 1]
prediction["hour"] = np.select(conditions, choices, default=0)

prediction["month"] = prediction["time"].dt.month
prediction["dayofweek"] = prediction["time"].dt.dayofweek
prediction["day"] = prediction["time"].dt.day
prediction = prediction.drop(columns=["time"]).drop_duplicates()
prediction = prediction.merge(stations, how="cross")
prediction["lat"] = (prediction["lat"] - min_lat) / (max_lat - min_lat)
prediction["long"] = (prediction["long"] - min_long) / (max_long - min_long)


# prediction["hour"] = prediction["hour"] / 23
# prediction = prediction[features].drop_duplicates()
prediction["month"] = prediction["month"] / 12
prediction["dayofweek"] = prediction["dayofweek"] / 6
# print(difference)
prediction["class"] = model.predict(prediction[features])

In [161]:
plenty_bikes = prediction.loc[prediction["class"] == 0]
lack_bikes = prediction.loc[prediction["class"] == 2]
print(plenty_bikes.shape, lack_bikes.shape)


(105, 14) (100, 14)
